In [1]:
!nvidia-smi


Wed May 15 16:25:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# PEGASUS Model
#https://huggingface.co/ArinaRomashova/summarisation-pegasus-pubmed
model_ckpt = "ArinaRomashova/summarisation-pegasus-pubmed"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/225 [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("ccdv/pubmed-summarization")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [12]:
from datasets import load_dataset

# Load the PubMed summarization dataset
dataset = load_dataset("ccdv/pubmed-summarization")

# Select a subset of the dataset
subset_size = 10000  # Specify the size of the subset you want
subset_dataset_train = dataset["train"].shuffle(seed=42).select(range(subset_size))

# Print the size of the subset dataset
print("Subset dataset size:", len(subset_dataset_train))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Subset dataset size: 10000


In [13]:
subset_dataset_test = dataset["test"].shuffle(seed=42).select(range(600))

In [14]:
subset_dataset_validation = dataset["validation"].shuffle(seed=42).select(range(600))

In [15]:
from datasets import Dataset, DatasetDict

# Combine articles and abstracts from each subset into a single list
merged_articles = (
    subset_dataset_train['article'] +
    subset_dataset_validation['article'] +
    subset_dataset_test['article']
)
merged_abstracts = (
    subset_dataset_train['abstract'] +
    subset_dataset_validation['abstract'] +
    subset_dataset_test['abstract']
)

# Create a merged dataset
merged_dataset = DatasetDict({
    'train': Dataset.from_dict({'article': merged_articles[:len(subset_dataset_train)], 'abstract': merged_abstracts[:len(subset_dataset_train)]}),
    'validation': Dataset.from_dict({'article': merged_articles[len(subset_dataset_train):len(subset_dataset_train)+len(subset_dataset_validation)], 'abstract': merged_abstracts[len(subset_dataset_train):len(subset_dataset_train)+len(subset_dataset_validation)]}),
    'test': Dataset.from_dict({'article': merged_articles[-len(subset_dataset_test):], 'abstract': merged_abstracts[-len(subset_dataset_test):]})
})

# Print the merged dataset
print(merged_dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 600
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 600
    })
})


In [16]:
dataset=merged_dataset

In [17]:
# print("Dataset version:", specific_dataset.info.version)
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 600
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 600
    })
})

In [18]:
split_lengths = [len(dataset[split])for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nArticle:")

print(dataset["test"][1]["article"])

print("\nAbstract:")

print(dataset["test"][1]["abstract"])

Split lengths: [10000, 600, 600]
Features: ['article', 'abstract']

Article:
de ridder , vanneste , and focquaert address concerns relating to the definition of a ced and the distinction between treatment and enhancement . 
 they raise a number of problems with the treatment - enhancement distinction and suggest that we need to ask  whether we are prepared to change the definition of health used by the medical devices directive. there are in fact three questions raised here . 
 the first is whether our proposal requires a robust characterization of the treatment - enhancement distinction , the second is whether our suggestions , as we intended them , actually involve changing the definition of heath implicit in the mdd ( there is no explicit definition ) , and the third is whether the mere inclusion of ceds within the mdd will have the de facto effect of changing the implicit regulatory concept of  health or 
 treatment. in response to the first question , we would maintain that it is 

In [19]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['abstract'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [20]:
dataset_pt = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [21]:
dataset_pt["train"]

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [22]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [23]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [24]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["test"],
                  eval_dataset=dataset_pt["validation"])

In [25]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=37, training_loss=2.1856320613139384, metrics={'train_runtime': 415.0257, 'train_samples_per_second': 1.446, 'train_steps_per_second': 0.089, 'total_flos': 1682430159568896.0, 'train_loss': 2.1856320613139384, 'epoch': 0.9866666666666667})

In [26]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [27]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-27-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [29]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'article', column_summary= 'abstract'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:25<00:00,  5.15s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018043,0.0,0.017263,0.018053


In [30]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [31]:

## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [32]:
#Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [36]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["article"]

reference = dataset["test"][0]["abstract"]

pipe = pipeline("summarization", model="ArinaRomashova/summarisation-pegasus-pubmed",tokenizer=tokenizer)

##
print("Article:")
print(sample_text)


print("\nReference Abstract:")
print(reference)




Article:
the birth of a premature infant has long been documented as a stressful event for parents . the premature birth of an infant and the following neonatal intensive care cause psychological distress and can have a traumatizing effect on parents . the neonatal intensive care unit ( nicu ) environment has the potential to exacerbate stress for parents of infants admitted to the nicu . 
 mothers have typically been found to have higher levels of distress than fathers and they experience significant levels of stress and depression in the early postpartum period . maternal stress can have deleterious effects on mother - infant interaction , particularly on mothers abilities to form an attachment to their baby . 
 maternal stress diminishes the mother 's ability to be sensitive to the infant 's cues and be responsive in interacting with the infant . participating in infant care influences the maternal feelings in a positive direction . 
 finding of a study has shown that a mother who i

In [50]:
# print("\nModel Abstract:")
# print(pipe(sample_text, **gen_kwargs)[0]["Abstract_text"])

In [48]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ArinaRomashova/summarisation-pegasus-pubmed")
max_length = tokenizer.model_max_length

# Truncate the input text if it exceeds the max token length
encoded_input = tokenizer(sample_text, truncation=True, max_length=max_length, return_tensors='pt')

# Decode back to text to ensure proper truncation
truncated_text = tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)

print("Truncated Article:")
print(truncated_text)

Truncated Article:
the birth of a premature infant has long been documented as a stressful event for parents. the premature birth of an infant and the following neonatal intensive care cause psychological distress and can have a traumatizing effect on parents. the neonatal intensive care unit ( nicu ) environment has the potential to exacerbate stress for parents of infants admitted to the nicu. mothers have typically been found to have higher levels of distress than fathers and they experience significant levels of stress and depression in the early postpartum period. maternal stress can have deleterious effects on mother - infant interaction, particularly on mothers abilities to form an attachment to their baby. maternal stress diminishes the mother's ability to be sensitive to the infant's cues and be responsive in interacting with the infant. participating in infant care influences the maternal feelings in a positive direction. finding of a study has shown that a mother who is deni

In [49]:
from transformers import pipeline

# Define the generation kwargs
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}

# Sample input data
sample_text = dataset["test"][0]["article"]
reference = dataset["test"][0]["abstract"]

# Initialize the summarization pipeline with the correct model and tokenizer
pipe = pipeline("summarization", model="ArinaRomashova/summarisation-pegasus-pubmed", tokenizer="ArinaRomashova/summarisation-pegasus-pubmed")

# Print the sample article
print("Article:")
print(sample_text)

# Print the reference abstract
print("\nReference Abstract:")
print(reference)

# Generate the summary and print the model's abstract
try:
    model_output = pipe(truncated_text, **gen_kwargs)

    # Print the model output to understand its structure
    print("\nModel Output:")
    print(model_output)

    # Check the structure of the output and print the abstract
    print("\nModel Abstract:")
    if model_output and isinstance(model_output, list) and len(model_output) > 0:
        if 'summary_text' in model_output[0]:
            print(model_output[0]['summary_text'])
        else:
            print("Unexpected output structure:", model_output[0])
    else:
        print("No output returned or unexpected output structure.")
except IndexError as e:
    print(f"Index Error: {e}")
except Exception as e:
    # Catch any other exceptions that might occur
    print(f"An unexpected error occurred: {e}")


Article:
the birth of a premature infant has long been documented as a stressful event for parents . the premature birth of an infant and the following neonatal intensive care cause psychological distress and can have a traumatizing effect on parents . the neonatal intensive care unit ( nicu ) environment has the potential to exacerbate stress for parents of infants admitted to the nicu . 
 mothers have typically been found to have higher levels of distress than fathers and they experience significant levels of stress and depression in the early postpartum period . maternal stress can have deleterious effects on mother - infant interaction , particularly on mothers abilities to form an attachment to their baby . 
 maternal stress diminishes the mother 's ability to be sensitive to the infant 's cues and be responsive in interacting with the infant . participating in infant care influences the maternal feelings in a positive direction . 
 finding of a study has shown that a mother who i